### 这样便是init最普通的用法了。但init其实不是实例化一个类的时候第一个被调用 的方法。当使用 Persion(name, age) 这样的表达式来实例化一个类时，最先被调用的方法 其实是 new 方法。

### 二、new 方法是什么？
new方法接受的参数虽然也是和init一样，但init是在类实例创建之后调用，而 new方法正是创建这个类实例的方法。

In [71]:
class Person(object):
    """Silly Person"""
 
    def __init__(self, name, age):
        self.name = name
        self.age = age
 
    def __str__(self):
        return '<Person: %s(%s)>' % (self.name, self.age)
 
if __name__ == '__main__':
    piglei = Person('piglei', 24)
    print (piglei)

<Person: piglei(24)>


In [72]:
class Person1(object):
    """Silly Person"""
 
    def __new__(cls, name, age):
        print ('__new__ called.')
        return super(Person1, cls).__new__(cls)  #return super(Person, cls).__new__(cls, name, age) This is python 2.7 version
 
    def __init__(self, name, age):
        print ('__init__ called.')
        self.name = name
        self.age = age
 
    def __str__(self):
        return '<Person: %s(%s)>' % (self.name, self.age)
 
if __name__ == '__main__':
    piglei = Person1('piglei', 24)
    print (piglei)

__new__ called.
__init__ called.
<Person: piglei(24)>


###   通过运行这段代码，我们可以看到，new方法的调用是发生在init之前的。其实当 你实例化一个类的时候，具体的执行逻辑是这样的：

1.p = Person(name, age)
2.首先执行使用name和age参数来执行Person类的new方法，这个new方法会 返回Person类的一个实例（通常情况下是使用 super(Persion, cls).new(cls, … …) 这样的方式），
3.然后利用这个实例来调用类的init方法，上一步里面new产生的实例也就是 init里面的的 self
所以，init 和 new 最主要的区别在于：
1.init 通常用于初始化一个新实例，控制这个初始化的过程，比如添加一些属性， 做一些额外的操作，发生在类实例被创建完以后。它是实例级别的方法。
2.new 通常用于控制生成一个新实例的过程。它是类级别的方法。
但是说了这么多，new最通常的用法是什么呢，我们什么时候需要new？

小编推荐一个学python的学习qun 740,3222,34
无论你是大牛还是小白，是想转行还是想入行都可以来了解一起进步一起学习！裙内有开发工具，很多干货和技术资料分享！
三、new 的作用
依照Python官方文档的说法，new方法主要是当你继承一些不可变的class时(比如int, str, tuple)， 提供给你一个自定义这些类的实例化过程的途径。还有就是实现自定义的metaclass。
首先我们来看一下第一个功能，具体我们可以用int来作为一个例子：
假如我们需要一个永远都是正数的整数类型，通过集成int，我们可能会写出这样的代码。

def __init__(self, value):
        super(PositiveInteger, self).__init__(self, abs(value))
 
i = PositiveInteger(-3)
print i
但运行后会发现，结果根本不是我们想的那样，我们任然得到了-3。这是因为对于int这种 不可变的对象，我们只有重载它的new方法才能起到自定义的作用。
这是修改后的代码：

def __new__(cls, value):
        return super(PositiveInteger, cls).__new__(cls, abs(value))
 
i = PositiveInteger(-3)
print i
通过重载new方法，我们实现了需要的功能。
另外一个作用，关于自定义metaclass。其实我最早接触new的时候，就是因为需要自定义 metaclass，但鉴于篇幅原因，我们下次再来讲python中的metaclass和new的关系。

四、用new来实现单例
事实上，当我们理解了new方法后，我们还可以利用它来做一些其他有趣的事情，比如实现 设计模式中的 单例模式(singleton) 。
因为类每一次实例化后产生的过程都是通过new来控制的，所以通过重载new方法，我们 可以很简单的实现单例模式。

def __new__(cls):
        # 关键在于这，每一次实例化的时候，我们都只会返回这同一个instance对象
        if not hasattr(cls, 'instance'):
            cls.instance = super(Singleton, cls).__new__(cls)
        return cls.instance
 
obj1 = Singleton()
obj2 = Singleton()
 
obj1.attr1 = 'value1'
print obj1.attr1, obj2.attr1
print obj1 is obj2
输出结果：

True
可以看到obj1和obj2是同一个实例。

In [89]:
class Singleton(object):
    def __new__(cls):
        print(f'cls = {id(cls)}')
        if  not hasattr(cls, 'instance'):
            cls.instance = super(Singleton, cls).__new__(cls)
        return cls.instance
    def __init__(self):
        print(f'self = {id(self)}')
    
instance = Singleton()
print(f'instance = {instance}')
print('*' * 30)
# instance1 = Singleton()
# print(instance1)
# print(id(Singleton))
Singleton.__dict__

cls = 93960518439192
self = 139717532082696
instance = <__main__.Singleton object at 0x7f1285de6208>
******************************


mappingproxy({'__module__': '__main__',
              '__new__': <staticmethod at 0x7f1285da9ef0>,
              '__init__': <function __main__.Singleton.__init__(self)>,
              '__dict__': <attribute '__dict__' of 'Singleton' objects>,
              '__weakref__': <attribute '__weakref__' of 'Singleton' objects>,
              '__doc__': None,
              'instance': <__main__.Singleton at 0x7f1285de6208>})